<a href="https://colab.research.google.com/github/Paymanshus/ML-Sem-V/blob/master/ML_J046_Lab12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time series analysis of UCI Time series data using machine learning models

## Objectives
- Download from Repository data
- Perform Time series analysis on given dataset
- Select correct architecture for single step multi-output and multi-step prediction
- List hyper-parameters used 


In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget -O AirQualityUCI.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
!unzip AirQualityUCI.zip
df = pd.read_excel('/content/AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']])
df.replace(to_replace=-200.0, value=0.0, inplace=True)
wv = df['NMHC(GT)']
bad_wv = wv == -200.0
wv[bad_wv] = 0.0

--2020-11-03 22:21:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1543989 (1.5M) [application/x-httpd-php]
Saving to: ‘AirQualityUCI.zip’

AirQualityUCI.zip   100%[===================>]   1.47M  3.22MB/s    in 0.5s    

2020-11-03 22:21:58 (3.22 MB/s) - ‘AirQualityUCI.zip’ saved [1543989/1543989]

Archive:  AirQualityUCI.zip
  inflating: AirQualityUCI.csv       
  inflating: AirQualityUCI.xlsx      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [3]:
target = 'NO2(GT)'
target

'NO2(GT)'

In [4]:
df.corr()[['NO2(GT)']]

,NO2(GT)
CO(GT),0.723154
PT08.S1(CO),0.284508
NMHC(GT),0.099541
C6H6(GT),0.402581
PT08.S2(NMHC),0.334108
NOx(GT),0.795888
PT08.S3(NOx),-0.440202
NO2(GT),1.000000
PT08.S4(NO2),0.010185
PT08.S5(O3),0.439057


In [5]:
df.head()
df = df[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)','PT08.S2(NMHC)', 'NOx(GT)','NO2(GT)','PT08.S5(O3)']]

In [6]:
df.head()

,CO(GT),PT08.S1(CO),NMHC(GT),PT08.S2(NMHC),NOx(GT),NO2(GT),PT08.S5(O3)
0,2.6,1360.00,150,1045.50,166.0,113.0,1267.50
1,2.0,1292.25,112,954.75,103.0,92.0,972.25
2,2.2,1402.00,88,939.25,131.0,114.0,1074.00
3,2.2,1375.50,80,948.25,172.0,122.0,1203.25
4,1.6,1272.25,51,835.50,131.0,116.0,1110.00


In [7]:
X = df.drop(target,axis=1)
y = df[[target]]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y)
value = y_test

In [9]:
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(7017, 6)
(2340, 6)
(7017, 1)
(2340, 1)


In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.fit_transform(x_train)
scaler.fit(x_test)
x_test = scaler.fit_transform(x_test)
scaler.fit(y_train)
y_train = scaler.fit_transform(y_train)
scaler.fit(y_test)
y_test = scaler.fit_transform(y_test)

In [12]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
print (x_train.shape)
print (x_test.shape)

(7017, 6, 1)
(2340, 6, 1)


In [13]:
print (y_train.shape)
print (y_test.shape)

(7017, 1)
(2340, 1)


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
def RNN_MODEL():
    # have to convert to 3D for feeding the data
    regressor = Sequential()
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    regressor.compile(optimizer = 'adam',loss = 'mean_squared_error',metrics=['accuracy'])
    return regressor

In [15]:
regressor = RNN_MODEL()

In [16]:
regressor.fit(x_train,y_train,epochs = 50)

Epoch 1/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0255 - accuracy: 0.1761
Epoch 2/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0218 - accuracy: 0.1764
Epoch 3/50
220/220 [==============================] - 3s 13ms/step - loss: 0.0198 - accuracy: 0.1764
Epoch 4/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0177 - accuracy: 0.1759
Epoch 5/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0168 - accuracy: 0.1763
Epoch 6/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0158 - accuracy: 0.1760
Epoch 7/50
220/220 [==============================] - 3s 11ms/step - loss: 0.0155 - accuracy: 0.1759
Epoch 8/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0156 - accuracy: 0.1761
Epoch 9/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0145 - accuracy: 0.1766
Epoch 10/50
220/220 [==============================] - 2s 11ms/step - loss: 0.0137 - accura

In [17]:
regressor.evaluate(x_test,y_test)

74/74 [==============================] - 0s 3ms/step - loss: 0.0055 - accuracy: 0.1731


[0.005491585936397314, 0.17307692766189575]

In [20]:
y_pred = regressor.predict(x_test)
y_pred

array([[0.36700216],
       [0.30441678],
       [0.4042767 ],
       ...,
       [0.35582736],
       [0.543862  ],
       [0.37139517]], dtype=float32)

In [21]:
output = scaler.inverse_transform(y_pred)
real_output = []
for item in output:
    real_output.append((item[0]))

In [22]:
real_output

[119.606,
 99.20943,
 131.75377,
 117.747475,
 96.216805,
 12.16423,
 35.047348,
 131.62624,
 150.74393,
 134.40228,
 86.3525,
 62.61335,
 70.27029,
 12.705978,
 113.561935,
 13.406221,
 120.93978,
 10.015153,
 193.54517,
 113.82172,
 11.263205,
 100.58376,
 160.27014,
 93.76996,
 131.62546,
 9.160087,
 137.08557,
 58.38185,
 164.64325,
 155.1576,
 102.66082,
 129.61978,
 126.301414,
 119.66799,
 65.15808,
 72.72359,
 17.802633,
 137.88304,
 97.178246,
 112.10977,
 129.79253,
 51.05113,
 68.4517,
 116.107155,
 187.35825,
 125.453415,
 200.78374,
 11.334636,
 88.1995,
 102.15418,
 142.13834,
 84.27433,
 14.0216875,
 94.04365,
 36.72589,
 31.250061,
 98.278755,
 154.21228,
 136.09984,
 130.71935,
 127.2901,
 122.08968,
 76.733604,
 158.19319,
 127.47085,
 127.92026,
 11.811188,
 109.050415,
 14.405932,
 11.9118395,
 69.942,
 136.78214,
 151.53784,
 120.69056,
 107.95451,
 110.00569,
 146.39479,
 102.43844,
 113.93151,
 113.84945,
 122.28896,
 139.86896,
 133.07631,
 145.75987,
 47.11852,

In [23]:
value['predicted'] = np.array(real_output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
value['predicted'][:5]

2717    119.606003
7496     99.209427
8970    131.753769
1063    117.747475
8798     96.216805
Name: predicted, dtype: float32